In [10]:
import os
import sys

import pandas as pd
import stanza
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [11]:
sys.path.append('..')

from utils.preprocessing import dummy_function
dummy_function('')

Hello World


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
stanza.download('en')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


In [ ]:
file_path = '/content/drive/MyDrive/online_sexism_detection/01_data/edos_labelled_individual_annotations.csv'
df = pd.read_csv(file_path)
df = df[["text","annotator","label_sexist","split"]]
df.head(5)

,text,annotator,label_sexist,split
0,[USER] I wonder what keeps that witch looking ...,17,sexist,train
1,[USER] I wonder what keeps that witch looking ...,2,sexist,train
2,[USER] I wonder what keeps that witch looking ...,6,not sexist,train
3,"What do you guys think about female ""incels""? ...",17,not sexist,train
4,"What do you guys think about female ""incels""? ...",15,not sexist,train


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   text          60000 non-null  object
 1   annotator     60000 non-null  int64 
 2   label_sexist  60000 non-null  object
 3   split         60000 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.8+ MB


In [ ]:
df.rename(columns={'label_sexist': 'label'}, inplace=True)
df['label'] = df['label'].map({'sexist': 1, 'not sexist': 0})

In [ ]:
stanza.download('en')  # Ensure the model is downloaded if not already
nlp = stanza.Pipeline('en', processors='tokenize,lemma')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| lemma     | combined_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
/usr/local/lib/python3.10/dist-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded v

In [ ]:
df_subset = df.head(100)

In [ ]:
nlp = stanza.Pipeline('en', processors='tokenize,lemma,pos')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
/usr/local/lib/python3.10/dist-packages/stanza/models/pos/trainer.py:139: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed d

In [ ]:
# Function to clean text and remove specific strings
def clean_text(text):
    return text.replace('[USER]', '').strip()  # Remove '[USER]' and any leading/trailing whitespace

# Function to lemmatize text and return as a list of lemmas
def lemmatize_text_to_list(text):
    doc = nlp(text)
    lemmatized_words = [word.lemma for sentence in doc.sentences for word in sentence.words]
    return lemmatized_words  # Return as a list

# Clean the 'text' column
df_subset['cleaned_text'] = df_subset['text'].apply(clean_text)

# Apply the lemmatization function to the cleaned 'text' column
df_subset['lemmatized_list'] = df_subset['cleaned_text'].apply(lemmatize_text_to_list)

# Display the updated DataFrame with the new columns
print(df_subset[['text', 'cleaned_text', 'lemmatized_list']].head())

<ipython-input-65-5d5091db50d0>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['cleaned_text'] = df_subset['text'].apply(clean_text)


                                                text  \
0  [USER] I wonder what keeps that witch looking ...   
1  [USER] I wonder what keeps that witch looking ...   
2  [USER] I wonder what keeps that witch looking ...   
3  What do you guys think about female "incels"? ...   
4  What do you guys think about female "incels"? ...   

                                        cleaned_text  \
0  I wonder what keeps that witch looking quite y...   
1  I wonder what keeps that witch looking quite y...   
2  I wonder what keeps that witch looking quite y...   
3  What do you guys think about female "incels"? ...   
4  What do you guys think about female "incels"? ...   

                                     lemmatized_list  
0  [I, wonder, what, keep, that, witch, look, qui...  
1  [I, wonder, what, keep, that, witch, look, qui...  
2  [I, wonder, what, keep, that, witch, look, qui...  
3  [what, do, you, guy, think, about, female, ", ...  
4  [what, do, you, guy, think, about, female, ", ..

<ipython-input-65-5d5091db50d0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['lemmatized_list'] = df_subset['cleaned_text'].apply(lemmatize_text_to_list)


In [ ]:
stanza.download('en')  # Ensure the model is downloaded if not already
nlp = stanza.Pipeline('en', processors='tokenize,lemma,pos')

# Select the first 100 rows
df_subset = df

# Function to clean text
def clean_text(text):
    return text.replace('[USER]', '').strip()

# Function to lemmatize text and return as a list of lemmas
def lemmatize_text_to_list(text):
    doc = nlp(text)
    lemmatized_words = [word.lemma for sentence in doc.sentences for word in sentence.words]
    return ' '.join(lemmatized_words)  # Join lemmas into a single string

# Clean and lemmatize the 'text' column
df_subset['cleaned_text'] = df_subset['text'].apply(clean_text)
df_subset['lemmatized_text'] = df_subset['cleaned_text'].apply(lemmatize_text_to_list)


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
/usr/local/lib/python3.10/dist-packages/stanza/models/tokenization/trainer.py:82: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will 

In [ ]:
import plotly.express as px
# Step 1: Vectorize the text data using TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df_subset['lemmatized_text']).toarray()  # Convert sparse matrix to dense

# Step 2: Apply PCA to reduce dimensionality to 2D
pca = PCA(n_components=2)
principal_components = pca.fit_transform(X)

# Step 3: Create a DataFrame for PCA results
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
pca_df['label'] = df_subset['label'].values  # Add labels to the DataFrame
pca_df['index'] = pca_df.index  # Add index for hover information

# Step 4: Plot the PCA results using Plotly
fig = px.scatter(
    pca_df,
    x='PC1',
    y='PC2',
    color='label',
    hover_data={'index': True, 'label': True},  # Show index and label on hover
    title='PCA of TF-IDF Vectors',
    color_discrete_sequence=px.colors.qualitative.Vivid  # Optional color palette
)

# Update layout
fig.update_layout(
    xaxis_title='Principal Component 1',
    yaxis_title='Principal Component 2',
    legend_title='Label',
)

# Show the plot
fig.show()